In [84]:
"""
Input:  JPG Image
        Annotation files in XML format (Ground Truth Bounding Boxes)

Output: Training data for ObjectNess Model
        X_positive, Y_positive, X_negative, Y_negative (.npy files)
        
The aim of this code is to extract positive (containing object) and negative (background) samples from the image for 
training Objecness Scoring Model
"""

'\nInput:  JPG Image\n        Annotation files in XML format (Ground Truth Bounding Boxes)\n\nOutput: Training data for ObjectNess Model\n        X_positive, Y_positive, X_negative, Y_negative (.npy files)\n        \nThe aim of this code is to extract positive (containing object) and negative (background) samples from the image for \ntraining Objecness Scoring Model\n'

In [82]:
#Import all the packages
import cv2
import matplotlib.pyplot as plt
import numpy as np
import cv2
import numpy as np
import os
import xml.etree.ElementTree as ET

In [2]:
#image and annotation path

image_path = "Images/"
annotation_path = "Annotation/"

In [ ]:
#resize the image in 384x288

In [3]:
#Stride and Sliding Window 
stepSize = 8
(w_width, w_height) = (64, 64) # window size

In [4]:
#iou function
#epsilon=1e-5
def get_iou(a, b, epsilon=0):
    """ Given two boxes `a` and `b` defined as a list of four numbers:
            [x1,y1,x2,y2]
        where:
            x1,y1 represent the upper left corner
            x2,y2 represent the lower right corner
        It returns the Intersect of Union score for these two boxes.

    Args:
        a:          (list of 4 numbers) [x1,y1,x2,y2]
        b:          (list of 4 numbers) [x1,y1,x2,y2]
        epsilon:    (float) Small value to prevent division by zero

    Returns:
        (float) The Intersect of Union score.
    """
    # COORDINATES OF THE INTERSECTION BOX
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    # AREA OF OVERLAP - Area where the boxes intersect
    width = (x2 - x1)
    height = (y2 - y1)
    # handle case where there is NO overlap
    if (width<0) or (height <0):
        return 0.0
    area_overlap = width * height

    # COMBINED AREA
    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    # RATIO OF AREA OF OVERLAP OVER COMBINED AREA
    iou = area_overlap / (area_combined+epsilon)
    return iou

In [24]:
# List to save all image crops and associated iou values calculated when compared to ground truth boxes
all_image_crops_with_ious = []

In [25]:
# Calculate all_image_crops_with_ious
for image in os.listdir(annotation_path):
    annotation_file = annotation_path+image
    image = cv2.imread(image_path+image.replace(".xml",".jpg"), cv2.IMREAD_GRAYSCALE) # your image path
    i=0
    for x in range(0, image.shape[1] - w_width , stepSize):
        for y in range(0, image.shape[0] - w_height, stepSize):
            #print(x, y, x+w_width, y+w_height)
            crop_img = image[y:y+w_height, x:x+w_width]
            #cv2.imwrite("Cropped_96/cropped_96_"+str(i)+".jpg",crop_img)
            #i=i+1
            et = ET.parse(annotation_file)
            element = et.getroot()
            element_objs = element.findall('object')
            #element_filename = element.find('filename').text
            #base_filename = os.path.join(data_path, element_filename)
            #print(base_filename)                               
            #img = cv2.imread(base_filename)
            #rows, cols = img.shape[:2] 
            #img_split = element_filename.strip().split('.jpg')
            for element_obj in element_objs:
                class_name = element_obj.find('name').text # return name tag ie class of disease from xml file
                #print(class_name)
                obj_bbox = element_obj.find('bndbox')
                #print(obj_bbox)
                x1 = int(round(float(obj_bbox.find('xmin').text)))
                y1 = int(round(float(obj_bbox.find('ymin').text)))
                x2 = int(round(float(obj_bbox.find('xmax').text)))
                y2 = int(round(float(obj_bbox.find('ymax').text)))
                #print(x1,y1,x2,y2)
                iou = get_iou([x1,y1,x2,y2],[x, y, x+w_width, y+w_height])
                if iou>0.0:
                    all_image_crops_with_ious.append([crop_img, iou])

In [80]:
#Refine ious for training the CNN
def get_refined_ious(iou):
    if iou>0.8 or iou==0.8:
        return 1.0
    elif iou>0.2 and iou<0.8:
        return iou
    else:
        return 0.0

In [59]:
get_refined_ious(0.81)

1.0

In [63]:
# Intialize list to save training examples and labels
X_positive= []
Y_positive= []
X_negative= []
Y_negative= []
pos = 0
neg = 0

In [64]:
# Append training data into lists
for data in all_image_crops_with_ious:
    if data[1]>0.2:
        cv2.imwrite("Positive/Positive_pipe_"+str(pos)+".jpg",data[0])
        X_positive.append(data[0])
        Y_positive.append(get_refined_ious(data[1]))
        pos = pos+1
    else:
        cv2.imwrite("Negative/Negative_pipe_"+str(neg)+".jpg",data[0])
        X_negative.append(data[0])
        Y_negative.append(0.0)
        neg = neg+1

In [69]:
#Number of positive Elements and Negative Elements
print("Positive: ", pos)
print("Negative: ", neg)

Positive:  79645
Negative:  187193


In [70]:
#Convert List into numpy array
X_positive= np.asarray(X_positive)
Y_positive= np.asarray(Y_positive)
X_negative= np.asarray(X_negative)
Y_negative= np.asarray(Y_negative)

In [74]:
X_positive.shape

(79645, 64, 64)

In [76]:
Y_positive.shape

(79645,)

In [77]:
X_negative.shape

(187193, 64, 64)

In [78]:
Y_negative.shape

(187193,)

In [79]:
#Save all the data
np.save("X_positive.npy",X_positive)
np.save("Y_positive.npy",Y_positive)
np.save("X_negative.npy",X_negative)
np.save("Y_negative.npy",Y_negative)